# Глава 2. Сверточные нейронные сети
# Часть 2. Приложения компьютерного зрения

## Зачем изучать компьютерное зрение?
Наиболее очевидный ответ заключается в том, что существует быстроразвивающие полезные приложения, полученных из этой области исследований. Вот лишь несколько из них:

* **Распознавание лиц:** Snapchat и Facebook используют алгоритмы распознавания лиц, чтобы применять фильтры и распознавать вас на фотографиях.


* **Поиск изображений:** Google Images использует контентные запросы для поиска релевантных изображений. Алгоритмы анализируют содержимое в изображении запроса и возвращают результаты на основе наиболее подходящего содержимого.


* **Игры и управление:** Отличным коммерческим продуктом в играх, использующим стереозрение, является Microsoft Kinect.


* **Наблюдение:** Камеры наблюдения широко распространены в общественных местах и используются для выявления подозрительных действий.


* **Биометрия:** отпечатки пальцев, радужная оболочка лица и идентификация лица остаются некоторыми распространенными методами биометрической идентификации.


* **Беспилотные автомобили:** Зрение остается основным источником информации для обнаружения дорожных знаков и огней и других визуальных функций.

Последние разработки в области нейронных сетей и подходов к глубокому обучению значительно повысили возможности современных систем визуального распознавания.

## 1. Классификация изображений

Задача классификации изображений звучит следующим образом: учитывая набор изображений, каждое из которых помечены одной категорией, нас просят предсказать эти категории для нового набора тестовых изображений и измерить точность прогноза.

Существует множество проблем, связанных с этой задачей, в том числе изменение точки обзора, изменение масштаба, внутриклассовая вариация, деформация изображения, доступность объекта на изображении (окклюзия), условия освещения и фоновый беспорядок.

* Задача Cats vs Dogs (Keras)
* Домашняя задача
* Transfer Learning
* Задача Flowers (fastai)
* Задача Planets (fastai)
* Домашняя задача (Plants)

Литература
* ImageNet http://www.image-net.org/
* AlexNet (2012) http://www.image-net.org/challenges/LSVRC/2012/supervision.pdf
* ZFNet (2013) https://arxiv.org/pdf/1311.2901.pdf
* GoogleNet (2014) https://arxiv.org/pdf/1409.4842.pdf
* VGG (2014) https://arxiv.org/pdf/1409.1556.pdf
* ResNet (2015) https://arxiv.org/pdf/1512.03385.pdf
* DenseNet (2016) https://arxiv.org/pdf/1608.06993.pdf

## 2. Детектирование объектов

Задача детектирования объектов на изображении обычно включает вывод ограничивающих рамок (bounding boxes) и меток (labels) для отдельных объектов. Это отличается от задачи классификации / локализации тем, что применяет классификацию и локализацию ко многим объектам, а не только к одному главному объекту на изображении.

Например, при обнаружении автомобилей вы должны обнаружить все автомобили на заданном изображении с помощью их ограничительных рамок (bounding boxes).

Если мы используем метод скользящего окна, например, как мы классифицируем и локализуем объект на изображении, нам нужно применить CNN ко многим различным сегментам (кропам) изображения. Поскольку CNN классифицирует каждую ситуацию как объект или фон, мы должны применить CNN к огромному количеству местоположений и масштабов, что очень дорого в вычислительном отношении!

**Какое есть решение?** По сути, мы можем превратить обнаружение объектов в задачу классификации изображений

Первой моделью, с которой все началось, была R-CNN (region-based сверточная нейронная сеть). В R-CNN мы сначала сканируем входное изображение на предмет возможных объектов, используя алгоритм выборочного поиска, генерирующий ~ 2000 вариантов областей. Затем мы запускаем CNN поверх каждого из этих вариантов. Наконец, мы берем выходные данные каждого CNN и подаем его в SVM для классификации региона и линейной регрессии для ужесточения ограничивающей рамки объекта.

![title](img/cnn31.jpeg)

По сути, мы превратили обнаружение объектов в задачу классификации изображений. Однако есть некоторые проблемы - обучение идет медленно, требуется много места на диске, и вывод также медленный.

Непосредственным потомком R-CNN является Fast R-CNN, который повышает скорость обнаружения за счет 2-х изменений: 
* 1) Выполнение извлечения признаков перед предложением областей, таким образом, работает только один CNN по всему изображению
* 2) Замена SVM слоем softmax

Bounding boxes также определяются линейной регрессией

![title](img/cnn32.jpeg)

Fast R-CNN показал гораздо лучшие результаты в отношении скорости, потому что по ходу обучается только одна CNN для всего изображения. Однако алгоритм выборочного поиска все еще занимает много времени для генерации интересующих регионов на изображении

Таким образом, появляется Faster R-CNN, которое теперь является канонической моделью для обнаружения объектов на основе глубокого обучения. 

В Faster R-CNN заменяется алгоритм медленного селективного поиска быстрой нейронной сетью, добавляя Region Propocal Network (RPN) для прогнозирования регионов поиска по признакам изображения.

RPN используется, чтобы решить задачу, «где» искать, чтобы уменьшить вычислительные требования всего процесса вывода. RPN быстро и эффективно сканирует каждое местоположение, чтобы оценить необходимость дальнейшей обработки в данном сегменте. Это достигается путем вывода k вариантов bounding boxes c вероятностью нахождения объекта.

![title](img/cnn33.jpeg)

Как только у нас появятся предложения по разбивочным сегментам, мы направим их прямо в то, что по сути является Fast R-CNN. Здесб находится слой пулинга, несколько полносвязных слоев и, наконец, слой классификации Softmax и регрессор для bounding boxes.

В целом, Faster R-CNN достиг гораздо лучших скоростей и более высокой точности. Стоит отметить, что, хотя будущие модели многое сделали для увеличения скорости обнаружения, немногим моделям все же удалось значительно опередить Faster R-CNN. **Другими словами, Faster R-CNN, возможно, не самый простой или быстрый способ обнаружения объектов, но он все еще один из лучших.**

Основные тенденции обнаружения объектов в последние годы сместились в сторону более быстрых и эффективных систем обнаружения.

Это стало видно в таких подходах, как You Only Look Once (YOLO), Single Shot MultiBox Detector (SSD) и region-based полностью сверточные сети (R-FCN), как шаг к совместному вычислению для всего изображения. 

Основное обоснование этих подходов состоит в том, чтобы избежать отдельных алгоритмов на их соответствующих подзадачах, поскольку это, как правило, увеличивает время обучения и может снизить точность сети.

### Далее YOLO и SSD
* Настройка своего YOLO или SSD object detection

Литература

* R-CNN https://www.cv-foundation.org/openaccess/content_cvpr_2014/papers/Girshick_Rich_Feature_Hierarchies_2014_CVPR_paper.pdf

* Fast R-CNN https://arxiv.org/pdf/1504.08083.pdf

* Faster R-CNN https://arxiv.org/pdf/1506.01497.pdf

* YOLO (2016) http://lanl.arxiv.org/pdf/1612.08242v1

* Single-shot SSD (2016) https://lanl.arxiv.org/pdf/1512.02325v5

* R-FCN (2016) https://lanl.arxiv.org/pdf/1605.06409v2

## 3. Трекинг объектов

Отслеживание объектов относится к процессу отслеживания определенного интересующего объекта или нескольких объектов в данной сцене. Он традиционно имеет приложения для видео и реального взаимодействия, где наблюдения производятся после первоначального обнаружения объекта. 

Мотивация - многие компании, например, Uber и Tesla смотрят в сторону автономных систем вождения. 

Методы отслеживания объектов можно разделить на две категории в соответствии с моделью наблюдения: 
* генеративный метод
* дискриминационный метод. 

Генеративный метод использует генеративную модель для описания видимых характеристик и минимизирует ошибку реконструкции для поиска объекта

Дискриминирующий метод может использоваться для разделения объекта и фона, его производительность более устойчива, и он постепенно становится основным методом отслеживания. 

**Дискриминационный метод** также называется **tracking-by-detection**, и к этой категории относится глубокое обучение. 

Чтобы добиться отслеживания по обнаружению (tracking-by-detection), мы обнаруживаем объекты-кандидаты для всех кадров и используем глубокое обучение для распознавания требуемого объекта из кандидатов. 

Существует два типа базовых сетевых моделей: 
* Stacked автоэнкодеры (SAE)
* Сверточная нейронная сеть (CNN)

![title](img/cnn34.jpeg)

Самая популярная глубокая сеть для отслеживания задач с использованием SAE - это Deep Learning Tracker, который предлагает предварительное обучение в автономном режиме и тонкую настройку сети. Процесс работает так:

* В офлайн режиме без учителя предварительно обучите **stacked denoising автоэнкодер** (SDAE), используя большой набор данных естественного изображения, чтобы получить общее признаковое представление объектов. Staked denosing авто-кодировщик может получить более надежную способность извлекать признаки, добавляя шум во входные изображения и восстанавливая исходные изображения.


* Далее кодирующая часть предварительно обученного автоэнкодера соединяется с классификатором. Затем используется разметку для точной настройки сети, которая может различать текущий объект и фон.
Deep Learning Tracker использует фильтр в качестве модели движения для создания возможных патчей (накладок) текущего (интересующего) кадра. 
Сеть классификации выводит оценки вероятности для этих пэтчей, а затем выбирает наиболее вероятные в качестве объекта.


* При обновлении модели, DLT использует метод ограниченния порога.

Из-за своего превосходства в классификации изображений и обнаружении объектов, CNN стала основной и в визуальном слежении.

Два популряных алгоритма трекинга на основе CNN - это **полностью сверточный трекер (FCNT)** и **многодоменный CNN (MD Net).** 


### Fully Convolutional Network Tracker (FCNT)


FCNT анализирует и использует преимущества модели VGG извлекать признаки из изображений, которая является предварительно обученной ImageNet

FCNT проектирует сеть выбора признаков для выбора наиболее релевантных карт свойств на уровнях conv4–3 и conv5–3 сети VGG. 

Затем, чтобы избежать переобучения, создаются два дополнительных канала (называемые SNet и GNet) для карт выбранных объектов из двух слоев отдельно. GNet собирает информацию о категории объекта (более глубокие карты conv5-3, описывающую семантику и описыв. категории, семантические признаки), в то время как SNet отличает объект от фона с похожим внешним видом (conv4-3, дискриманантные признаки, внутриклассовые различия)

![title](img/cnn35.jpeg)

### Multi-domain Network

В отличие от идеи FCNT, MD Net использует все последовательности видео для отслеживания движений в них. 

Так как объект одного класса в этом видео может быть фоном в другом видео, в MD Net предлагается идея мультидомена, чтобы различать объект и фон в каждом домене независимо. 

И домен представляет собой набор видео, которые содержат объект такого же типа.

Как показано ниже, MD Net делится на 2 части: общие слои и K ветвей доменных слоев. 

Каждая ветвь содержит бинарный классификационный слой с softmax, который используется для различия объекта и фона в каждом домене, а также общие слои, используемые совместно со всеми доменами для обеспечения общего представления.

![title](img/cnn36.jpeg)

В последние годы исследователи глубокого обучения пробовали разные способы адаптации к особенностям задачи визуального отслеживания. Было исследовано много направлений: применение других сетевых моделей, таких как:
* Recurrent Neural Net
* Deep Belief Net.

### Далее про Deep Belief Networks
(https://codeburst.io/deep-learning-deep-belief-network-fundamentals-d0dcfd80d7d4)

Литература

* SDAE https://papers.nips.cc/paper/5192-learning-a-deep-compact-image-representation-for-visual-tracking.pdf

* FCN tracker
https://www.semanticscholar.org/paper/Visual-Tracking-with-Fully-Convolutional-Networks-Wang-Ouyang/328a523c0fc9a3e6ea9615fe7b9a36037a1b4f63

* MD CNN https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Nam_Learning_Multi-Domain_Convolutional_CVPR_2016_paper.pdf

## 4. Семантическая сегментация

Сегментация изображения - центральная задача машинного зрения, суть которой заключается в разделение целого изображения на группы пикселей, которые можно отнести к отдному классу.

В частности, семантическая сегментация пытается семантически понять роль каждого пикселя в изображении (например, это автомобиль, мотоцикл или какой-то другой класс). Например, на картинке нижу, кроме распознавания человека, также дороги, автомобилей, деревьев и т. д. 

Мы также должны разграничить границы каждого объекта. Поэтому, в отличие от классификации, нам нужны плотные попиксельные прогнозы на основе моделей.

![title](img/cnn37.png)

Как и в случае с другими задачами компьютерного зрения, CNN имели огромный успех в решении проблем сегментации.

**Полностью сверточными сетями (FCN)** UC Berkeley (https://arxiv.org/pdf/1411.4038.pdf), которые популяризировали сквозную архитектуру CNN для плотных попиксельных предсказаний без каких-либо полносвязных слоев. 

Это позволило создавать карты сегментации для изображений любого размера, а также было намного быстрее по сравнению с подходом классификации патчей.

Почти все последующие подходы к семантической сегментации приняли эту парадигму.

![title](img/cnn38.jpeg)

Однако остается одна проблема: свертки при исходном разрешении изображения вычислять очень накладко.

Чтобы преодолеть это ограничение, FCN использует понижающую (**downsampling**) и повышающую (**upsampling**) дискретизацию внутри сети. Слой **понижающей дискретизации** известен как полосатая свертка (**striped convolution**), в то время как слой **повышающей дискретизации** известен как транспонированная свертка (**transposed convolution**).

Несмотря на upsampling и downsampling слои, FCN создает карты достаточно грубой (coarse) сегментации из-за потери информации во время пулинга (субдискретизации).

**SegNet** - это более эффективная архитектура с точки зрения памяти, чем FCN, которая использует макспулинг и структуру атоэнкодера (кодировщика-декодировщика).

В SegNet вводятся skip connections из карт признаков с более высоким разрешением, чтобы улучшить разрешение после upsampling/downsampling.

Недавние исследования в области семантической сегментации в значительной степени основаны на полностью сверточных сетях, таких как:

* Dilated Convolutions (https://arxiv.org/pdf/1511.07122.pdf)

* DeepLab (https://arxiv.org/pdf/1412.7062.pdf)

* RefineNet (https://arxiv.org/pdf/1611.06612.pdf)


## 5. Сегментация экземпляра

Помимо семантической сегментации, сегментация экземпляров сегментирует различные экземпляры классов, например, маркируя 5 автомобилей 5 разными цветами. 

При классификации обычно используется изображение с одним объектом в качестве фокуса, и задача состоит в том, чтобы сказать, что это за изображение. 

Но чтобы сегментировать экземпляры, нам нужно решать гораздо более сложные задачи. 

Мы видим сложные изображения с несколькими перекрывающимися объектами и разными фонами, и мы не только классифицируем эти разные объекты, но и определяем их границы, различия и отношения друг к другу

![title](img/cnn39.jpeg)

### Mask R-CNN

До сих пор мы видели, как использовать возможности CNN, чтобы эффективно определять местоположение различных объектов на изображении с помощью ограничительных рамок (bounding boxes).

Можем ли мы расширить эти подходы, чтобы найти точные пиксели каждого объекта вместо просто ограничивающих рамок?

Эта проблема сегментации экземпляра решается с использованием архитектуры, известной как **Mask R-CNN**.

Так же, как Fast R-CNN и Faster R-CNN, базовая интуиция Mask R-CNN проста. 

Учитывая, что Faster R-CNN так хорошо работает для обнаружения объектов, можем ли мы расширить его для выполнения сегментации на уровне пикселей?

Mask R-CNN делает это, расширяя Faster R-CNN за счет дополнительной ветки, которая выводит двоичную маску, которая сообщает, является ли данный пиксель частью объекта. 

Блок является полностью сверточной сетью FCN поверх карты признаков изображения из CNN. Принимая в качестве входных данных карту признаков CNN, сеть выводит матрицу с 1 во всех местах, где пиксель принадлежит объекту, и 0 в других местах.

Это называется **бинарной маской**.

![title](img/cnn40.jpeg)

Кроме того, при использовании без изменений в исходной архитектуре Faster R-CNN, регионы карты признаков, выбранные RoIPool (Region of Interests Pool), слегка смещены относительно областей исходного изображения. 

Поскольку сегментация изображения требует специфичности на уровне пикселей, в отличие от ограничивающих рамок, это, естественно, приводит к неточностям.

Mask R-CNN решает эту проблему, настраивая RoIPool для более точного выравнивания, используя метод, известный как RoIAlign (Region of Interests Align). 

По сути, RoIAlign использует билинейную интерполяцию, чтобы избежать ошибок при округлении, что приводит к неточностям в детектировании и сегментации.

После того как эти маски сгенерированы, Mask R-CNN объединяет их с классификациями и ограничивающими рамками из Faster R-CNN, чтобы создать такие удивительно точные сегментации:

![title](img/cnn41.jpeg)

## 6. Генерация изображений

Генеративные состязательные модели

## 7. Стилизация изображений

В основе стилизации изображений лежит ключевая идея, что возможно разделить представление стиля и представления контента в CNN, обученные в рамках задачи распознавания изображений

Следуя этой концепции, для решении задачи стилизации может быть использована предобученная сверточная нейронная сеть (CNN), например, VGG 16, для переноса стиля от одного изображения на другое.

Это делается путем определения функции потерь, которая пытается минимизировать различия между изображением контента, изображением стиля и сгенерированным изображением.

![title](img/cnn42.jpeg)

* Задача transer learning на основе VGG 16

Литература:

https://heartbeat.fritz.ai/20-minute-masterpiece-4b6043fdfff5

https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-neural-style-transfer-ef88e46697ee

## 8. Идентификация позы

![title](img/cnn45.jpg)

Идентификация позы относится к методам компьютерного зрения, которые обнаруживают скелетон человека на изображении. Алгоритм оценивает, где находятся ключевые суставы тела. При этом не обеспечивает идентификацию самого человека

PoseNet может использоваться для оценки одной или нескольких поз, что означает, что существует версия алгоритма, которая может обнаруживать только одного человека на изображении / видео, и одна версия, которая может обнаруживать несколько человек на изображении.

Почему есть две версии? 

Детектор позы для одного человека работает быстрее и проще, но требует только одного объекта, присутствующего на изображении (подробнее об этом позже). 

Сначала мы расскажем об одиночной позе, потому что за ней легче следовать.

Верхнеуровнево идентификация позы происходит в два этапа:

* Входное RGB-изображение подается через сверточную нейронную сеть.

* Алгоритм декодирования используется для декодирования поз, оценок достоверности поз, позиций ключевых точек и оценок достоверности ключевых точек из выходных данных модели.

**Pose** на самом высоком уровне PoseNet - это объект позы, который содержит список ключевых точек и показатель достоверности на уровне экземпляра для каждого обнаруженного человека.

![title](img/cnn43.png)

* **Оценка уверенности (pose confidence score)** - определяет общую уверенность в оценке позы. Он колеблется между 0,0 и 1,0. Его можно использовать, чтобы скрыть позы, которые не считаются достаточно уверенно распознанными.

* **Ключевая точка (keypoint)** - оцениваемая часть позы человека, такая как нос, правое ухо, левое колено, правая ступня и т. д. Она содержит как позицию, так и оценку достоверности ключевой точки. В настоящее время PoseNet обнаруживает 17 ключевых точек, показанных на следующей диаграмме ниже.

* **Оценка достоверности ключевой точки (keypoint CS)** - это определяет уверенность в том, что оценочная позиция ключевой точки является точной. Он колеблется между 0,0 и 1,0. Его можно использовать, чтобы скрыть ключевые точки, которые не считаются достаточно достоверными

* **Положение ключевой точки (keypoint position)** - 2D координаты x и y в исходном входном изображении, где обнаружена ключевая точка.

![title](img/cnn44.png)

Одна важная деталь, на которую следует обратить внимание, заключается в том, что исследователи обучили ResNet MobileNet версии PoseNet. 

Хотя модель ResNet обладает более высокой точностью, ее большой размер и множество слоев делают время загрузки страницы и время вывода менее чем идеальным для любых приложений в real-time. 

В данный момент большинство использует модель MobileNet, так как она разработана для работы на мобильных устройствах.

**Задача обучения детектировать действие**

Литература:
    
https://medium.com/tensorflow/real-time-human-pose-estimation-in-the-browser-with-tensorflow-js-7dd0bc881cd5